# Свёрточные нейросети и POS-теггинг

POS-теггинг - определение частей речи (снятие частеречной неоднозначности)

In [137]:
# Если Вы запускаете ноутбук на colab или kaggle,
# выполните следующие строчки, чтобы подгрузить библиотеку dlnlputils:

# !git clone https://github.com/arturburiev/stepik-dl-nlp.git
# !pip install -r stepik-dl-nlp/requirements.txt
# import sys; sys.path.append('./stepik-dl-nlp')

In [138]:
# !pip install pyconll
# !pip install spacy_udpipe

In [1034]:
#!g1.1
%load_ext autoreload
%autoreload 2

import sys; sys.path.append('./stepik-dl-nlp')

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

import numpy as np

import pyconll

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset

import dlnlputils
from dlnlputils.data import tokenize_corpus, build_vocabulary, \
    character_tokenize, pos_corpus_to_tensor, POSTagger
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

init_random_seed()

[nltk_data] Downloading package omw-1.4 to /home/jupyter/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Загрузка текстов и разбиение на обучающую и тестовую подвыборки

In [189]:
# # Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# !wget -O ./stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
# !wget -O ./stepik-dl-nlp/datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

In [1030]:
#!c1.8
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
full_train = pyconll.load_from_file('./stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('./stepik-dl-nlp/datasets/ru_syntagrus-ud-dev.conllu')

In [1031]:
#!c1.8
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [1032]:
#!c1.8
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [1033]:
#!c1.8
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [1035]:
#!g1.1
train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, max_doc_freq=1.0, min_count=5, pad_word='<PAD>')
print("Количество уникальных символов", len(char_vocab))
print(list(char_vocab.items())[:10])

Количество уникальных символов 142
[('<PAD>', 0), (' ', 1), ('о', 2), ('е', 3), ('а', 4), ('т', 5), ('и', 6), ('н', 7), ('.', 8), ('с', 9)]


In [1036]:
#!g1.1
UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}
label2id

{'<NOTAG>': 0,
 'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'PUNCT': 13,
 'SCONJ': 14,
 'SYM': 15,
 'VERB': 16,
 'X': 17}

In [1037]:
#!c1.8
START_WORD_ID = len(char_vocab)
END_WORD_ID = len(char_vocab)

In [1038]:
#!c1.8
train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN, start_word_id = START_WORD_ID, end_word_id = END_WORD_ID)
train_dataset = TensorDataset(train_inputs, train_labels)

test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN, start_word_id = START_WORD_ID, end_word_id = END_WORD_ID)
test_dataset = TensorDataset(test_inputs, test_labels)

In [1039]:
#!c1.8
train_inputs[1][:5]

tensor([[142,  38,   4,  25,   4,  11,  19,   7,   6,  13, 142,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0],
        [142,   2,  23,  11,   4,   9,   5,   7,   2,  22,   2, 142,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0],
        [142,  17,  16,  10,   4,  12,  11,   3,   7,   6,  20, 142,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0],
        [142,   9,  12,  20,  21,   6, 142,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0],
        [142,  40,   3,  15,   3,   7, 142,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0]])

In [1040]:
#!c1.8
train_labels[1]

tensor([ 8,  1,  8,  8, 12, 12,  4,  8,  1, 13, 16,  2,  8,  3,  3, 13, 16,  2,
         8,  2,  8,  5,  3, 10, 16,  2,  8,  8,  2,  8, 13,  8, 13, 13,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [271]:
# #!c1.8
# torch.save(train_inputs, './stepik-dl-nlp/datasets/train_inputs_dataset_postag.pt')
# torch.save(train_labels, './stepik-dl-nlp/datasets/train_labels_dataset_postag.pt')
# torch.save(test_inputs, './stepik-dl-nlp/datasets/test_inputs_dataset_postag.pt')
# torch.save(test_labels, './stepik-dl-nlp/datasets/test_labels_dataset_postag.pt')

# import pickle

# with open('./stepik-dl-nlp/datasets/char_vocab_postag.pt', 'wb') as f:
#     pickle.dump(char_vocab, f)
    
# with open('./stepik-dl-nlp/datasets/label2id_postag.pt', 'wb') as f:
#     pickle.dump(label2id, f)

In [733]:
# #!c1.8
# train_inputs = torch.load('./stepik-dl-nlp/datasets/train_inputs_dataset_postag.pt')
# train_labels = torch.load('./stepik-dl-nlp/datasets/train_labels_dataset_postag.pt')
# test_inputs = torch.load('./stepik-dl-nlp/datasets/test_inputs_dataset_postag.pt')
# test_labels = torch.load('./stepik-dl-nlp/datasets/test_labels_dataset_postag.pt')

# train_dataset = TensorDataset(train_inputs, train_labels)
# test_dataset = TensorDataset(test_inputs, test_labels)

# import pickle

# with open('./stepik-dl-nlp/datasets/char_vocab_postag.pt', 'rb') as f:
#     char_vocab = pickle.load(f)

# # with open('./stepik-dl-nlp/datasets/label2id_postag.pt', 'rb') as file:
# #     label2id = pickle.load(file)

## Вспомогательная свёрточная архитектура

In [1041]:
#!c1.8
class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernels=3, conv_layer=nn.Conv1d, dropout=0.0, dilations=1):
        super().__init__()
        layers = []
        for i in range(layers_n):
            l_kernel = kernels if isinstance(kernels, int) else kernels[i]
            l_dilation = dilations if isinstance(dilations, int) else dilations[i]
            l_padding = (l_kernel - 1) * l_dilation//2
            
            layers.append(nn.Sequential(
                conv_layer(features_num, features_num, l_kernel, padding=l_padding, dilation=l_dilation),
                nn.Dropout(dropout),
#                 nn.BatchNorm1d(features_num, affine=True),
                nn.LeakyReLU(),
            ))
            
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers:
            x = x + layer(x)
        return x

## Предсказание частей речи на уровне отдельных токенов

In [ ]:
#!c1.8
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [ ]:
#!g1.1
single_token_model = SingleTokenPOSTagger(len(char_vocab), len(label2id), embedding_size=64, layers_n=3, kernel_size=3, dropout=0.3)
print('Количество параметров', sum(np.product(t.shape) for t in single_token_model.parameters()))

In [ ]:
#!g1.1
(best_val_loss,
 best_single_token_model) = train_eval_loop(single_token_model,
                                            train_dataset,
                                            test_dataset,
                                            F.cross_entropy,
                                            lr=5e-3,
                                            epoch_n=10,
                                            batch_size=64,
                                            device='cuda',
                                            early_stopping_patience=5,
                                            max_batches_per_epoch_train=500,
                                            max_batches_per_epoch_val=100,
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                       factor=0.5,
                                                                                                                       verbose=True))

In [145]:
#!c1.8
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# torch.save(best_single_token_model.state_dict(), './models/single_token_pos.pth')

In [ ]:
#!c1.8
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# single_token_model.load_state_dict(torch.load('./models/single_token_pos.pth'))

In [ ]:
#!g1.1
single_token_model = best_single_token_model

In [ ]:
#!g1.1
train_pred = predict_with_model(single_token_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(single_token_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

## Предсказание частей речи на уровне предложений (с учётом контекста)

In [1042]:
#!c1.8
class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [1043]:
#!g1.1

# Изначальные настройки
# EPOCH_N = 10
# EMB_SIZE = 64
# DROPOUT = 0.3
# LAYERS_N = 3
# KERNELS = 3
# DILATIONS = 1
# BATCH_NORMALIZATION = False

# Итоговые настройки
EPOCH_N = 100
EMB_SIZE = 128
DROPOUT = 0.6
LAYERS_N = 5
KERNELS = [3, 5, 5, 5, 5]
DILATIONS = [1, 2, 2, 3, 3]
BATCH_NORMALIZATION = False

sentence_level_model = SentenceLevelPOSTagger(len(char_vocab) + 1, len(label2id), embedding_size=EMB_SIZE,
                                              single_backbone_kwargs=dict(layers_n=LAYERS_N, kernels=KERNELS, dropout=DROPOUT, dilations=DILATIONS),
                                              context_backbone_kwargs=dict(layers_n=LAYERS_N, kernels=KERNELS, dropout=DROPOUT, dilations=DILATIONS))

print(f"Настройки: EPOCH_N = {EPOCH_N}, EMB_SIZE = {EMB_SIZE}, LAYERS_N = {LAYERS_N}, KERNELS = {KERNELS}, DILATIONS = {DILATIONS}, DROPOUT = {DROPOUT}, BATCH_NORMALIZATION = {BATCH_NORMALIZATION}")
print('Количество параметров:', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

Настройки: EPOCH_N = 100, EMB_SIZE = 128, LAYERS_N = 5, KERNELS = [3, 5, 5, 5, 5], DILATIONS = [1, 2, 2, 3, 3], DROPOUT = 0.6, BATCH_NORMALIZATION = False
Количество параметров: 775570


### Взвешивание классов

In [737]:
#!c1.8
from sklearn.utils.class_weight import compute_class_weight
class_weights=compute_class_weight('balanced',np.unique(train_labels),train_labels.numpy().ravel())
class_weights=torch.tensor(class_weights,dtype=torch.float)

In [1044]:
#!g1.1
import sys; sys.path.append('./stepik-dl-nlp')
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

torch.backends.cudnn.benchmark=True
torch.backends.cudnn.deterministic=False 

criterion_func = F.cross_entropy
# criterion_func = nn.CrossEntropyLoss(weight=class_weights,reduction='mean').cuda()

(best_val_loss,
 best_sentence_level_model) = train_eval_loop(sentence_level_model,
                                              train_dataset,
                                              test_dataset,
                                              criterion_func,
                                              lr=5e-3,
                                              epoch_n=EPOCH_N,
                                              batch_size=64,
                                              device='cuda',
                                              early_stopping_patience=5,
                                              max_batches_per_epoch_train=500,
                                              max_batches_per_epoch_val=100,
                                              lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                         factor=0.5,
                                                                                                                         verbose=True))

[nltk_data] Downloading package omw-1.4 to /home/jupyter/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Эпоха 0
Эпоха: 384 итераций, 50.15 сек
Среднее значение функции потерь на обучении 0.2512350682518445
Среднее значение функции потерь на валидации 0.049622557411837104
Новая лучшая модель!

Эпоха 1
Эпоха: 384 итераций, 48.44 сек
Среднее значение функции потерь на обучении 0.03677669228394128
Среднее значение функции потерь на валидации 0.032596804939284185
Новая лучшая модель!

Эпоха 2
Эпоха: 384 итераций, 48.55 сек
Среднее значение функции потерь на обучении 0.02897957497771131
Среднее значение функции потерь на валидации 0.025268050897313227
Новая лучшая модель!

Эпоха 3
Эпоха: 384 итераций, 48.62 сек
Среднее значение функции потерь на обучении 0.025338300154544413
Среднее значение функции потерь на валидации 0.021916042801250916
Новая лучшая модель!

Эпоха 4
Эпоха: 384 итераций, 48.72 сек
Среднее значение функции потерь на обучении 0.02348013773977679
Среднее значение функции потерь на валидации 0.020656871812252123
Новая лучшая модель!

Эпоха 5
Эпоха: 384 итераций, 48.75 сек
Средне

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# torch.save(best_sentence_level_model.state_dict(), './models/sentence_level_pos.pth')

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# sentence_level_model.load_state_dict(torch.load('./models/sentence_level_pos.pth'))

In [1045]:
#!g1.1
sentence_level_model = best_sentence_level_model

In [1046]:
#!g1.1
UNIQUE_TAGS = list(label2id.keys())
train_pred = predict_with_model(sentence_level_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             train_labels.clone().detach())
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            test_labels.clone().detach())
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [00:17, 45.02it/s]                             


Среднее значение функции потерь на обучении 0.0047452752478420734


  2%|▏         | 5/278.3125 [00:00<00:06, 42.07it/s]

              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.98      0.98      0.98     42552
         ADP       1.00      1.00      1.00     39344
         ADV       0.93      0.98      0.95     22448
         AUX       0.94      0.94      0.94      3531
       CCONJ       0.97      0.98      0.97     15168
         DET       0.95      0.95      0.95     10780
        INTJ       0.97      0.68      0.80        50
        NOUN       0.99      0.99      0.99    103538
         NUM       0.96      0.97      0.97      6764
        PART       0.98      0.94      0.96     13540
        PRON       0.97      0.95      0.96     18732
       PROPN       0.97      1.00      0.99     14858
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.95      0.91      0.93      8022
         SYM       0.99      1.00      0.99       420
        VERB       0.99      0.99      0.99     47753
           X       0.99    

279it [00:06, 45.35it/s]                              


Среднее значение функции потерь на валидации 0.010289125144481659
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1574439
         ADJ       0.95      0.94      0.95     14793
         ADP       1.00      1.00      1.00     13717
         ADV       0.90      0.95      0.92      7714
         AUX       0.92      0.91      0.92      1391
       CCONJ       0.96      0.98      0.97      5672
         DET       0.93      0.88      0.91      4265
        INTJ       0.80      0.50      0.62        24
        NOUN       0.98      0.98      0.98     36238
         NUM       0.91      0.94      0.93      2119
        PART       0.97      0.89      0.93      5117
        PRON       0.95      0.94      0.95      7444
       PROPN       0.91      0.97      0.94      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.91      0.88      0.90      2859
         SYM       0.97      1.00      0.98        62
        VERB   

## Применение полученных теггеров и сравнение

In [ ]:
#!g1.1
single_token_pos_tagger = POSTagger(sentence_level_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN, start_word_id = START_WORD_ID, end_word_id = END_WORD_ID)

test_sentences = [
    'Мама мыла раму.',
    'Косил косой косой косой.',
    'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
    'Сяпала Калуша с Калушатами по напушке.',
    'Пирожки поставлены в печь, мама любит печь.',
    'Ведро дало течь, вода стала течь.',
    'Три да три, будет дырка.',
    'Три да три, будет шесть.',
    'Сорок сорок'
]
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

for sent_tokens, sent_tags in zip(test_sentences_tokenized, single_token_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

In [1047]:
#!g1.1
sentence_level_pos_tagger = POSTagger(sentence_level_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN, start_word_id = START_WORD_ID, end_word_id = END_WORD_ID)

test_sentences = [
    'Мама мыла раму.',
    'Косил косой косой косой.',
    'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
    'Сяпала Калуша с Калушатами по напушке.',
    'Пирожки поставлены в печь, мама любит печь.',
    'Ведро дало течь, вода стала течь.',
    'Три да три, будет дырка.',
    'Три да три, будет шесть.',
    'Сорок сорок'
]
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

for sent_tokens, sent_tags in zip(test_sentences_tokenized, sentence_level_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00,  1.99it/s]                     


мама-NOUN мыла-NOUN раму-NOUN

косил-VERB косой-NOUN косой-NOUN косой-NOUN

глокая-ADJ куздра-NOUN штеко-ADV будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-NOUN

ведро-ADV дало-NOUN течь-NOUN вода-NOUN стала-VERB течь-NOUN

три-NUM да-PART три-NUM будет-VERB дырка-NOUN

три-NUM да-PART три-NUM будет-VERB шесть-NUM

сорок-NUM сорок-NUM



## Свёрточный модуль своими руками

In [ ]:
class MyConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(in_channels * kernel_size, out_channels) / (in_channels * kernel_size),
                                   requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(out_channels), requires_grad=True)
    
    def forward(self, x):
        """x - BatchSize x InChannels x SequenceLen"""

        batch_size, src_channels, sequence_len = x.shape        
        if self.padding > 0:
            pad = x.new_zeros(batch_size, src_channels, self.padding)
            x = torch.cat((pad, x, pad), dim=-1)
            sequence_len = x.shape[-1]

        chunks = []
        chunk_size = sequence_len - self.kernel_size + 1
        for offset in range(self.kernel_size):
            chunks.append(x[:, :, offset:offset + chunk_size])

        in_features = torch.cat(chunks, dim=1)  # BatchSize x InChannels * KernelSize x ChunkSize
        in_features = in_features.permute(0, 2, 1)  # BatchSize x ChunkSize x InChannels * KernelSize
        out_features = torch.bmm(in_features, self.weight.unsqueeze(0).expand(batch_size, -1, -1)) + self.bias.unsqueeze(0).unsqueeze(0)
        out_features = out_features.permute(0, 2, 1)  # BatchSize x OutChannels x ChunkSize
        return out_features

In [ ]:
sentence_level_model_my_conv = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                                      single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d),
                                                      context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model_my_conv.parameters()))

In [ ]:
(best_val_loss,
 best_sentence_level_model_my_conv) = train_eval_loop(sentence_level_model_my_conv,
                                                      train_dataset,
                                                      test_dataset,
                                                      F.cross_entropy,
                                                      lr=5e-3,
                                                      epoch_n=10,
                                                      batch_size=64,
                                                      device='cuda',
                                                      early_stopping_patience=5,
                                                      max_batches_per_epoch_train=500,
                                                      max_batches_per_epoch_val=100,
                                                      lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                                 factor=0.5,
                                                                                                                                 verbose=True))

In [ ]:
train_pred = predict_with_model(best_sentence_level_model_my_conv, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(best_sentence_level_model_my_conv, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))